<a href="https://colab.research.google.com/github/kazimianec/TARIC/blob/master/Training_TRCtransformersSBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch


In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
print(torch.cuda.device_count())

1


In [4]:
url_TARIC_C2_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C2_descriptions.csv"
url_TARIC_C4_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C4_descriptions.csv"
url_TARIC_C6_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C6_descriptions.csv"
url_TARIC_C246_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C246_descriptions_CONCAT.csv"


In [5]:
import pandas as pd 
import io 
  
df_C2_descriptions = pd.read_csv(url_TARIC_C2_descrptions) 
df_C4_descriptions = pd.read_csv(url_TARIC_C4_descrptions)
df_C6_descriptions = pd.read_csv(url_TARIC_C6_descrptions)
df_C246_descriptions = pd.read_csv(url_TARIC_C246_descrptions)

In [6]:
for col in df_C246_descriptions.columns: 
    print(col)

CODE_2
CODE_4
CODE_6
goods_CODE_6
Description_46
Description_246
Description_46_concat
Description_246_concat


In [7]:
#sentences = df['CLASS_Description'].tolist()

sentences_C2 = df_C2_descriptions['Description_2'].tolist()
sentences_C4 = df_C4_descriptions['Description_4'].tolist()
sentences_C6 = df_C6_descriptions['Description_6'].tolist()

sentences_C246 = df_C246_descriptions['Description_246'].tolist()
sentences_C46 = df_C246_descriptions['Description_46'].tolist()

sentences_C246_concat = df_C246_descriptions['Description_246_concat'].tolist()
sentences_C46_concat = df_C246_descriptions['Description_46_concat'].tolist()


In [8]:
!pip install sentence_transformers

     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 1.8MB 28.8MB/s 
     |████████████████████████████████| 1.2MB 53.8MB/s 
     |████████████████████████████████| 890kB 43.9MB/s 
     |████████████████████████████████| 2.9MB 56.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=ee18fc2110301937a30788c0fbc6a5360e245f31f2c2f26d9f8bbc84dfde8ca1
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9a8887e572c13402d9116f051b1da7105237712655ba50b6b908308063c8c561
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [9]:
from sentence_transformers import SentenceTransformer, util


**SBERT**
Models can be loaded from SBERT , full list:
https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0


In [10]:
#model = SentenceTransformer('average_word_embeddings_glove.840B.300d')
#model = SentenceTransformer('paraphrase-distilroberta-base-v1') --no
#model= SentenceTransformer('msmarco-distilroberta-base-v2') --no
model = SentenceTransformer('stsb-bert-large')  ##maybe
##model = SentenceTransformer('stsb-roberta-large') --no
#model= SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
#model= SentenceTransformer('LaBSE')  # interesting
#model = SentenceTransformer('quora-distilbert-multilingual') ## not ok

#model = SentenceTransformer('average_word_embeddings_glove.840B.300d')

100%|██████████| 1.24G/1.24G [00:40<00:00, 30.8MB/s]


In [36]:
sentences = sentences_C246
sentences = [element.lower() for element in sentences]

sentences[1682]+=' nail polishing makeup'
print(sentences[1682])

essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; manicure or pedicure preparations nail polishing makeup


In [37]:

embeddings = model.encode(sentences, convert_to_tensor=True)
#embeddings_246_concat = model.encode(sentences_246_concat, convert_to_tensor=True)
#embeddings_46_concat = model.encode(sentences_46_concat, convert_to_tensor=True)
#embeddings_246 = model.encode(sentences_246, convert_to_tensor=True)
#embeddings_46 = model.encode(sentences_46, convert_to_tensor=True)

In [38]:
TAROC_cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
print(TAROC_cosine_scores.shape)

top_k=5
top_results = torch.topk(TAROC_cosine_scores, k=top_k)
print(TAROC_cosine_scores.shape)
print(TAROC_cosine_scores)
print(top_results.values)
#print(top_results.indices)
#print(sentences[0])
#print(sentences[40])
#y = torch.ones(TAROC_cosine_scores.shape)*0.8
#torch.where(TAROC_cosine_scores > y, TAROC_cosine_scores, 0)

torch.Size([5209, 5209])
torch.Size([5209, 5209])
tensor([[1.0000, 0.8130, 0.7635,  ..., 0.2059, 0.2538, 0.1940],
        [0.8130, 1.0000, 0.9174,  ..., 0.1528, 0.2301, 0.2185],
        [0.7635, 0.9174, 1.0000,  ..., 0.1840, 0.2500, 0.2369],
        ...,
        [0.2059, 0.1528, 0.1840,  ..., 1.0000, 0.9151, 0.8263],
        [0.2538, 0.2301, 0.2500,  ..., 0.9151, 1.0000, 0.8647],
        [0.1940, 0.2185, 0.2369,  ..., 0.8263, 0.8647, 1.0000]])
tensor([[1.0000, 0.8130, 0.7807, 0.7635, 0.7430],
        [1.0000, 0.9491, 0.9174, 0.8130, 0.7189],
        [1.0000, 0.9190, 0.9174, 0.7635, 0.7538],
        ...,
        [1.0000, 0.9151, 0.8263, 0.7457, 0.6404],
        [1.0000, 0.9151, 0.8647, 0.7212, 0.6591],
        [1.0000, 0.8647, 0.8263, 0.6879, 0.6367]])


In [39]:
description = ["Vyriškos kelnės"]

description = ["Мужские джинсы"]
description = ["Beef fillet"]

description = ["Jautienos filė"]

description = ["Лак для маникюра"]

description = ["Nail polish"]

description = ["Кресла офисные"]

description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]

description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["I Love Makeup - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]



description = ["папиросы"]
description = ["I Love Makeup  - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Beef fillet"]
description = ["Гoвядина филе морож"]
description = ["Мужские джинсы"]
description = ["Фанера"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Vineer on kihiline materjal, mis valmistatakse õhukeste puitlehtede – spoonide – kokkuliimimise teel."] 
description = ["офисные стулья деревянные дубовые"] 
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"] #SUPER matches S3
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"] # not
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"] ## bullshit is both cases - bert and S3
description = ["Jolly Jumper Pashmama Nursing Cover - Tan"]
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["I Love Makeup  - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"]


description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Beef fillet"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Phone Protector Cover Case"]
description = ["I Love Makeup  - Nail Polish"]

embeddings_description = model.encode(description, convert_to_tensor=True )

In [40]:
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings_description)
cosine_scores.shape
values,indices = torch.transpose(cosine_scores,0,1).topk(10)
print(values, indices)
indices.numpy()[0][0]

print(values.numpy()[0][0], sentences[indices.numpy()[0][0]])
print(values.numpy()[0][1], sentences[indices.numpy()[0][1]])
print(values.numpy()[0][2], sentences[indices.numpy()[0][2]])
print(values.numpy()[0][3], sentences[indices.numpy()[0][3]])
print(values.numpy()[0][4], sentences[indices.numpy()[0][4]])

print(sentences[1682])




tensor([[0.5621, 0.5126, 0.5063, 0.4814, 0.4687, 0.4382, 0.4089, 0.4089, 0.4089,
         0.4089]]) tensor([[1682, 1713, 1680, 1712, 1681, 1684, 1701, 1699, 1700, 1698]])
0.562145 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; manicure or pedicure preparations nail polishing makeup
0.51262975 soap, organic surface-active agents, washing preparations, lubricating preparations, artificial waxes, prepared waxes, polishing or scouring preparations, candles and similar articles, modelling pastes, 'dental waxes' and dental preparations with a basis of plaster; artificial waxes and prepared waxes; other
0.5063278 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), includin

In [33]:
df_C246_descriptions.iloc[indices.numpy()[0],:]

,CODE_2,CODE_4,CODE_6,goods_CODE_6,Description_46,Description_246,Description_46_concat,Description_246_concat
1713,34,4,90,3404900000 80,Artificial waxes and prepared waxes; Other,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P...",Artificial waxes and prepared waxes; Other,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P..."
1680,33,4,10,3304100000 80,Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...",Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."
1712,34,4,20,3404200000 80,Artificial waxes and prepared waxes; Of poly(o...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P...",Artificial waxes and prepared waxes; Of poly(o...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P..."
1681,33,4,20,3304200000 80,Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...",Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."
1682,33,4,30,3304300000 80,Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...",Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."
1684,33,4,91,3304990000 80,Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...",Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."
1701,34,1,30,3401300000 80,Soap; organic surface-active products and prep...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P...",Soap; organic surface-active products and prep...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P..."
1699,34,1,11,3401190000 80,Soap; organic surface-active products and prep...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P...",Soap; organic surface-active products and prep...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P..."
1700,34,1,20,3401200000 80,Soap; organic surface-active products and prep...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P...",Soap; organic surface-active products and prep...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P..."
1698,34,1,11,3401110000 80,Soap; organic surface-active products and prep...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P...",Soap; organic surface-active products and prep...,"SOAP, ORGANIC SURFACE-ACTIVE AGENTS, WASHING P..."


**TRAINING**

In [22]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
t_model = model

#Define your train examples. You need more than just two examples...
#train_examples = [
#                  InputExample(texts=['Nail polish makeup', 'manicure or pedicure preparations'], label=1.),
#                  InputExample(texts=['Nail polish makeup', 'essential oils and resinoids; perfumery, cosmetic or toilet preparations'], label=1.),
#                  InputExample(texts=['Nail polish makeup', 'soap, organic surface-active agents, washing preparations, lubricating preparations, artificial waxes, prepared waxes, polishing or scouring preparations, candles and similar articles, modelling pastes'], label=0.)]

train_examples = [
                  InputExample(texts=['Nail polish makeup', 'manicure or pedicure preparations'], label=1.),
                  #InputExample(texts=['Nail polish makeup', 'essential oils and resinoids; perfumery, cosmetic or toilet preparations'], label=1.),
                  InputExample(texts=['Nail polish makeup', 'waxes'], label=0.),
                  InputExample(texts=['Nail polish makeup', 'lips'], label=0.),
                  InputExample(texts=['Nail polish makeup', 'eye'], label=0.)]



#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=1)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
t_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

In [23]:
t_embeddings = t_model.encode(sentences, convert_to_tensor=True)

In [24]:
t_embeddings_description = t_model.encode(description, convert_to_tensor=True )

cosine_scores = util.pytorch_cos_sim(t_embeddings, t_embeddings_description)
cosine_scores.shape
values,indices = torch.transpose(cosine_scores,0,1).topk(10)
print(values, indices)
indices.numpy()[0][0]

print(values.numpy()[0][0], sentences[indices.numpy()[0][0]])
print(values.numpy()[0][1], sentences[indices.numpy()[0][1]])
print(values.numpy()[0][2], sentences[indices.numpy()[0][2]])
print(values.numpy()[0][3], sentences[indices.numpy()[0][3]])
print(values.numpy()[0][4], sentences[indices.numpy()[0][4]])



tensor([[0.5126, 0.5063, 0.4814, 0.4687, 0.4474, 0.4382, 0.4359, 0.4089, 0.4089,
         0.4089]]) tensor([[1713, 1680, 1712, 1681, 1682, 1684, 1707, 1699, 1701, 1700]])
0.51262975 soap, organic surface-active agents, washing preparations, lubricating preparations, artificial waxes, prepared waxes, polishing or scouring preparations, candles and similar articles, modelling pastes, 'dental waxes' and dental preparations with a basis of plaster; artificial waxes and prepared waxes; other 
0.5063278 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; lip make-up preparations 
0.48139337 soap, organic surface-active agents, washing preparations, lubricating preparations, artificial waxes, prepared waxes, polishing or scouring preparations, candles and similar articles, modelling pastes, 'dent